In [1]:
import numpy as np
import matplotlib
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
import joblib
from sklearn.svm import SVC

In [2]:
fueldata=pd.read_csv('PWR_UOX-MOX_BigDataFrame-SF-GSRC.csv')

In [3]:
for columna in ["Unnamed: 0","reactorType"]:
    fueldata= fueldata.loc[:, fueldata.columns != columna] 
fueldata
X=fueldata
Y=fueldata[['fuelType']]

for columna in ['BU', 'CT', 'IE','fuelType']:
    X= X.loc[:, X.columns != columna] 
    X = X.loc[:, X.columns != columna]

In [4]:
from sklearn.preprocessing import StandardScaler
X_s = StandardScaler()
X_escalada=  X_s.fit_transform(X.values)

In [5]:
from sklearn import preprocessing
Encoder = preprocessing.LabelEncoder()
Y['fuelType']=Encoder.fit_transform(Y['fuelType']).reshape(-1,1)


f:\python\python37_64\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
_conjunto_de_train_X,_conjunto_de_test_X,_conjunto_de_train_Y,_conjunto_de_test_Y=train_test_split(X_escalada,Y,test_size=0.2,random_state=128)#conjunto_X_de20000,conjunto_Y_de20000
print(_conjunto_de_train_X.shape,_conjunto_de_test_X.shape)
print(_conjunto_de_train_Y.shape,_conjunto_de_test_Y.shape)

(1057117, 283) (264280, 283)
(1057117, 1) (264280, 1)


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
print(datetime.now())
fold= KFold(n_splits=5, shuffle=True, random_state=127)
print(datetime.now())

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.svm import SVC
print(datetime.now())
row_SVM={"MODEL":"SVR","C":100,"Epsilon":0.1,"Kernel":"linear","R2_fold1":0.0,"R2_fold2":0.0,"R2_fold3":0.0,"R2_fold4":0.0,"R2_fold5":0.0}
SVC =  SVC(max_iter=1000)
i=1
for train_index, test_index in fold.split(_conjunto_de_train_X,_conjunto_de_train_Y):
    X_train = _conjunto_de_train_X[train_index]
    X_test = _conjunto_de_train_X[test_index]
    y_train = _conjunto_de_train_Y["fuelType"].iloc[train_index].values
    y_test = _conjunto_de_train_Y["fuelType"].iloc[test_index].values
    SVC.fit(X_train, y_train)
    row_SVM["R2_fold"+str(i)] = accuracy_score(y_test, SVC.predict(X_test))
    print(datetime.now())
    i += 1

2020-08-25 19:07:34.034387


f:\python\python37_64\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


2020-08-25 19:15:30.689366


f:\python\python37_64\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


2020-08-25 19:23:35.671993


...

2020-08-25 19:31:38.514616


f:\python\python37_64\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


2020-08-25 19:39:40.245277


f:\python\python37_64\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


2020-08-25 19:47:53.243609


In [17]:
from sklearn.metrics import confusion_matrix

Vector_entrenamiento=SVC.predict(_conjunto_de_train_X)
Vector_prueba=SVC.predict(_conjunto_de_test_X)
matriz_entrenamiento=confusion_matrix(_conjunto_de_train_Y["fuelType"].values, Vector_entrenamiento)
matriz_prueba=confusion_matrix(_conjunto_de_test_Y["fuelType"].values, Vector_prueba)
matriz_prueba=matriz_prueba.ravel()
matriz_entrenamiento=matriz_entrenamiento.ravel()
tn, fp, fn, tp





In [62]:
matriz_entrenamiento

array([425699,      0,      0, 631418], dtype=int64)

In [27]:
def metricas_clasificador(tn,fp,fn,tp,Tipo="entrenamiento"):
    Precision= 0.0+((tp+tn)/(tp+tn+fp+fn))*100
    Sensibilidad= 0.0+((tp)/(tp+fn))*100
    Especificidad= ((tn)/(tn+fp))*100
    VPP= ((tp)/(tp+fp))*100
    VPN= ((tn)/(tn+fn))*100
    MCC_1=np.float32(tp*tn-fp*fn)
    MCC_2 =np.float32(tp+fn)*np.float32(tp+fp)*np.float32(tn+fp)*np.float32(tn+fn)
    MCC=MCC_1/np.sqrt(MCC_2)
    print(Tipo)
    print("precision:",Precision)
    print("Sensibilidad:",Sensibilidad)
    print("Especificidad:",Especificidad)
    print("VPP:",VPP,"VPN",VPN,"MCC",MCC)

matriz_prueba=matriz_prueba.ravel()
matriz_entrenamiento=matriz_entrenamiento.ravel()
metricas_clasificador(matriz_entrenamiento[0],matriz_entrenamiento[1],matriz_entrenamiento[2],matriz_entrenamiento[3],"entrenamiento")
metricas_clasificador(matriz_prueba[0],matriz_prueba[1],matriz_prueba[2],matriz_prueba[3],"test")

entrenamiento
precision: 100.0
Sensibilidad: 100.0
Especificidad: 100.0
VPP: 100.0 VPN 100.0 MCC 1.0
test
precision: 100.0
Sensibilidad: 100.0
Especificidad: 100.0
VPP: 100.0 VPN 100.0 MCC 1.0


In [19]:
joblib.dump(SVC,"SVM_sin_ajuste.model")

['SVM_sin_ajuste.model']

In [7]:
from sklearn.feature_selection import RFECV
y = np.array(Y['fuelType'].values)
y = y.ravel()
_conjunto_de_train_X,_conjunto_de_test_X,_conjunto_de_train_Y,_conjunto_de_test_Y=train_test_split(X_escalada,y,test_size=0.2,random_state=128)
svc=SVC(kernel="linear", max_iter = 1000) 
print(datetime.now())
rfecv = RFECV(estimator=svc, step=1, cv=2,scoring='accuracy',n_jobs=1)
print(datetime.now())
rfecv.fit(_conjunto_de_train_X, _conjunto_de_train_Y)
print(datetime.now())
print("Optimal number of features : %d" % rfecv.n_features_)

2020-08-26 08:46:33.562092
2020-08-26 08:46:33.562092
2020-08-26 14:16:09.801766
Optimal number of features : 1


In [8]:
"""
tenemos que dejarlo corriendo en el SOMA, asi que hasta aqui no mas 
"""
rfecv = joblib.dump("SVC.model")

In [24]:
rfecv_features=[]
i=0
for bool, feature in zip(rfecv.get_support(), X.columns):
    if bool:
        print(i)
        rfecv_features.append(feature)
    i+=1
rfecv_features

17


['O16']

In [47]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
row_SVM={"MODEL":"SVR","C":100,"Epsilon":0.1,"Kernel":"linear","R2_fold1":0.0,"R2_fold2":0.0,"R2_fold3":0.0,"R2_fold4":0.0,"R2_fold5":0.0}
SVC_ajustado =  SVC(max_iter=1000)
i=1
fold_1= KFold(n_splits=5, shuffle=True, random_state=127)
for train_index, test_index in fold_1.split(_conjunto_de_train_X,_conjunto_de_train_Y):
    X_train = _conjunto_de_train_X[train_index,17]
    X_test = _conjunto_de_train_X[test_index,17]
    y_train = _conjunto_de_train_Y["fuelType"].iloc[train_index]
    y_test = _conjunto_de_train_Y["fuelType"].iloc[test_index]
    SVC_ajustado.fit(X_train.reshape(-1, 1), y_train)
    row_SVM["R2_fold"+str(i)] = accuracy_score(y_test, SVC_ajustado.predict(X_test.reshape(-1, 1)))
    print(datetime.now())
    i += 1

2020-08-26 14:44:39.727832
2020-08-26 14:44:40.224176
2020-08-26 14:44:40.739825
2020-08-26 14:44:41.254989
2020-08-26 14:44:41.759311


In [48]:
row_SVM

{'MODEL': 'SVR',
 'C': 100,
 'Epsilon': 0.1,
 'Kernel': 'linear',
 'R2_fold1': 1.0,
 'R2_fold2': 1.0,
 'R2_fold3': 1.0,
 'R2_fold4': 1.0,
 'R2_fold5': 1.0}

In [52]:
from sklearn.metrics import confusion_matrix
Vector_entrenamiento=SVC_ajustado.predict(_conjunto_de_train_X[:,17].reshape(-1, 1))
Vector_prueba=SVC_ajustado.predict(_conjunto_de_test_X[:,17].reshape(-1, 1))
matriz_entrenamiento=confusion_matrix(_conjunto_de_train_Y["fuelType"].values, Vector_entrenamiento)
matriz_prueba=confusion_matrix(_conjunto_de_test_Y["fuelType"].values, Vector_prueba)
matriz_prueba=matriz_prueba.ravel()
matriz_entrenamiento=matriz_entrenamiento.ravel()

In [53]:
def metricas_clasificador(tn,fp,fn,tp,Tipo="entrenamiento"):
    Precision= 0.0+((tp+tn)/(tp+tn+fp+fn))*100
    Sensibilidad= 0.0+((tp)/(tp+fn))*100
    Especificidad= ((tn)/(tn+fp))*100
    VPP= ((tp)/(tp+fp))*100
    VPN= ((tn)/(tn+fn))*100
    MCC_1=np.float32(tp*tn-fp*fn)
    MCC_2 =np.float32(tp+fn)*np.float32(tp+fp)*np.float32(tn+fp)*np.float32(tn+fn)
    MCC=MCC_1/np.sqrt(MCC_2)
    print(Tipo)
    print("precision:",Precision)
    print("Sensibilidad:",Sensibilidad)
    print("Especificidad:",Especificidad)
    print("VPP:",VPP,"VPN",VPN,"MCC",MCC)

matriz_prueba=matriz_prueba.ravel()
matriz_entrenamiento=matriz_entrenamiento.ravel()
metricas_clasificador(matriz_entrenamiento[0],matriz_entrenamiento[1],matriz_entrenamiento[2],matriz_entrenamiento[3],"entrenamiento")
metricas_clasificador(matriz_prueba[0],matriz_prueba[1],matriz_prueba[2],matriz_prueba[3],"test")

entrenamiento
precision: 100.0
Sensibilidad: 100.0
Especificidad: 100.0
VPP: 100.0 VPN 100.0 MCC 1.0
test
precision: 100.0
Sensibilidad: 100.0
Especificidad: 100.0
VPP: 100.0 VPN 100.0 MCC 1.0


In [54]:
matriz_entrenamiento

array([425699,      0,      0, 631418], dtype=int64)

In [60]:
Encoder.fit_transform(list(Encoder.classes_))

array([0, 1], dtype=int64)

In [61]:
list(Encoder.classes_)

['MOX', 'UOX']

In [64]:
joblib.dump(SVC,  'SVM_TypeFuel_2608.model')
joblib.dump(SVC_ajustado,  'SVM_selecciondeatributos_016_TypeFuel_2608.model')

['SVM_selecciondeatributos_016_TypeFuel_2608.model']